<a href="https://colab.research.google.com/github/dandemonium/sail-tutorials/blob/main/SAIL_exoplanet_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### $\underline{\rm Objectives}$:


1.   Identify exoplanetary transit signals in time-series photometry ("light curves") from NASA's Transiting Exoplanet Survey Satellite (TESS) mission.
2.   Estimate physical parameters of transiting exoplanetary systems from transit features.
3.   Gain familiarity with Python tools used by professional astronomers to collate, process, and analyze light curves from TESS (and *Kepler*/K2).

### $\underline{\rm Let's\ begin!}$

0.   Set up this Google Colab notebook:



In [ ]:
### Install Lightkurve and ipympl
%pip install -q lightkurve ipympl

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
import matplotlib.widgets
import lightkurve as lk, numpy as np, matplotlib.pyplot as plt, numpy.random as random, ipympl

In [ ]:
### Select a KELT transiting exoplanet system:
star_name = 'KELT-' + str(random.randint(1,27))
print('System: ' + star_name)


1.   Use the Lightkurve module to find 2-minute cadence light curves (LCs) from [MAST](https://archive.stsci.edu/). TESS observes each sector of sky for ~27 days; sectors overlap, so same target may appear in multiple sectors.

2.   Download LCs, excluding individual observations flagged as "bad", and store the high-quality data in a [$\texttt{LightCurveCollection()}$](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurveCollection.**html**) object.

In [ ]:
result = lk.search_lightcurve(star_name, mission='TESS', author='SPOC', exptime=120)
print(result)
lcs = result.download_all(quality_bitmask='hard')


3.   Estimate the observable transit parameters by plotting the stitched-together LCs and inspecting the transits: pan & zoom around the plot to estimate:

*    the duration $T$ and depth $\delta$ of the transits, and

*    the planet's ephemeris -- its orbital period $P$ and time of [one of the] transit center[s] $T_C$.



In [ ]:
combined_lc = lcs.stitch().remove_nans()  # combine into 1 LC & remove NaN values
combined_lc.scatter() #or .plot() or .errorbar()

4.   Check your guesstimated ephemeris by "phase-folding" the LCs with [$\texttt{LightCurve.fold()}$](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurve.fold.html), which performs the modular arithmetic for us:

   *   determines how many fractional orbits have occurred between $T_C$ and each observation by subtracting $T_C$ from the times of the observations, then divides result by the orbital period $P$.

   *   takes the result "modulo 1" to superimpose all observations on the same orbit.

then over-plot the in-transit data only, using [$\texttt{LightCurve.create_transit_mask()}$](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurve.create_transit_mask.html) to identify in-transit data.

In [ ]:
### replace values w/ your guesstimates here:
per_guess = 3.3 # days
tc_guess = 1494.133 # BTJD
dur_guess = 0.1 # days
depth_guess = 0.01 # fraction

# use BLS ephemeris to identify in-transit data
mask = combined_lc.create_transit_mask(period=per_guess, transit_time=tc_guess, duration=dur_guess)

# create phase-folded LCs
folded_lc = combined_lc.fold(period=per_guess, epoch_time=tc_guess)

# plot phase-folded LCs
bls_ax = folded_lc.scatter(label='full LC')
folded_lc[mask].scatter(ax=bls_ax, label='in-transit data')

5.   Get a more accurate ephemeris from the [Box Least Squares](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.periodogram.BoxLeastSquaresPeriodogram.from_lightcurve.html) periodogram ([Kovacs et al. 2002](https://ui.adsabs.harvard.edu/abs/2002A%26A...391..369K/abstract)). BLS is similar to a Fourier-transform based [Lomb-Scargle](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.periodogram.LombScarglePeriodogram.from_lightcurve.html) [periodogram](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurve.to_periodogram.html): whereas Lomb-Scargle periodograms search for sinusoidal ("sine wave"-looking) signals, BLS searches for "boxcar"-shaped transit signals.

> *Note: Google Colab does not support the backend needed to use the interactive version -- [interact_bls()](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurve.interact_bls.html) -- so we proceed the old-fashioned way.*

In [ ]:
### Uncomment next line if running in local Jupyter notebook:
#combined_lc.interact_bls()   # may need to pass: notebook_url='localhost:8888'

### Uncomment this section if running in Google Colab (or just want regular BLS),
### and fiddle with input arguments as necessary to find an accurate ephemeris:

# BLS inputs:
per_lower = 1.0   # shortest period to search
per_upper = 5.0   # longest period to search
n_per = 20000   # how many periods to search
ffactor = 1000   # controls the spacing between trial frequencies

per_arr = np.linspace(per_lower, per_upper, n_per)   # Create array of periods to search

# Run BLS, then plot, store, and print the results:
bls = combined_lc.to_periodogram(method = 'bls', period=per_arr, frequency_factor=ffactor)
bls.plot()

planet_b_period = bls.period_at_max_power
planet_b_tc = bls.transit_time_at_max_power
planet_b_dur = bls.duration_at_max_power
planet_b_depth = bls.depth_at_max_power

print("From BLS maximum power:")
print("Period (days): ", planet_b_period.value)
print("Tc (BTJD): ", planet_b_tc.value)
print("Transit duration (days): ", planet_b_dur.value)
print("Transit depth (fraction): ", planet_b_depth.value)

6.   Phase-fold the LC on this ephemeris to verify its accuracy. Tweak your BLS arguments as necessary to recover a precise and accurate ephemeris.

> *Note: In Google Colab, it may not be feasible to sample enough periods to determine a super-accurate ephemeris.*

In [ ]:
combined_lc.fold(period=planet_b_period, epoch_time=planet_b_t0).scatter()

# Investigating Further:


7.    Assuming your system has a circular orbit which we are viewing "edge-on," write a couple functions that take in the ephemeris, duration, and depth you determined in Steps 4-6 and return:
######    a) the planet-to-star radius ratio, $R_p/R_*$;
######    b) the relative size of the star, $R_*/a$ (which is also the "a priori" transit probability);
######    c) and the density of the star, $\rho_*$.
######    d) [Optional] If you want to determine $R_*$ so you can determine $a$, $R_p$, and $M_*$, jump to the "Appendix" at the bottom of this notebook. If you just want a decent estimate of $R_*$, you can grab one from [NASA's Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/).
---


8.   Besides any phase-folding issues, do you notice any strange features in the observed transits, or any coherent variations in the unphased light curves? If so, what do you think could explain these features?
---


9.   Do you see evidence for a secondary eclipse, when the planet goes behind the star?
######    a) Assuming a circular orbit, at which phase should secondary eclipse occur? At which time of secondary eclipse, Ts? How deep do you expect the eclipse to be?
######    b) Inspect LC plots to spot the secondary eclipse. Try "binning" the light curve, using the [LightCurve.bin()](https://lightkurve.github.io/lightkurve/reference/api/lightkurve.LightCurve.bin.html) function.
---

### 10) Want some challenges?
###### a) Query for the "Pi Mensae" planetary system, and see if you can excavate the transit signal from underneath some gnarly noise by masking out low-quality observations and "de-trending" any correlated noise with LightCurve.flatten().

###### b) Ask Dan for some examples of transit-like eclipsing binary stars that looked great when we identified them as transiting planets in ground-based KELT data, but show very bad behavior in their TESS light curves.
---

### 11) Want to learn more?

#####      a) The Lightkurve [GitHub repository](https://lightkurve.github.io/lightkurve/tutorials/index.html) has some excellent tutorials like this one (which was adapted from them, in fact)

#####      b) Consider taking AST 2050 (General Astronomy) with Jessica Rogers in the fall! It is an excellent class, and a prerequisite for both the stellar & exoplanetary astrophysics (PHYS 3561; Dan) and galactic astrophysics (3562; Alec Habig) courses, which are only offered in alternating fall semesters.
---



# Appendix A: Spectral Energy Distribution (SED) Fitting
> *Note: Does not seem to generate in-line plots.*

In [ ]:
### Let's try this SED fitting routine that Dan discovered two nights ago:
%pip install -q sedkit
from sedkit import SED, modelgrid as mg
import astropy.units as q
from dustmaps.config import config
config.reset()

1.   Query a database for the parallax (distance) measurement and the multi-wavelength flux measurements for your target. Grab the Gaia team's computed stellar parameters, too, to use as guesses in the SED fit.

In [ ]:
# Initialize SED structure, then grab 2MASS, WISE, and Gaia data
starsed = SED(name=star_name)
starsed.find_2MASS()
starsed.find_WISE()
starsed.find_Gaia(include=['parallax', 'photometry', 'teff', 'Lbol'])

2.    Perform an SED fit behind-the-scenes to determine the values for the stellar parameters ("param"s) that produce the best-fit SED model, and determine the uncertainties ("unc") on these parameters, which include:

   *   distance: between us and the target, in parsecs (1 pc = 206,265 au.)
   *   fbol: bolometric flux $F_{\rm bol}$ in "cgs" units (emitted power per unit surface area)   
   *   Lbol: bolometric luminosity $L_{\rm bol}$ in cgs units (emitted pwr/area)
   *   Lbol_sun: $L_{\rm bol}$ relative to Sun's luminosity $L_{\odot}$
   *   Teff: surface temperature $T_{\rm eff}$, in Kelvin (really: "effective" temperature of an equivalent blackbody)
   *   mass: relative to the Sun's mass $M_{\odot}$ ($1 M_{\odot} \sim 10^3$ Jupiter masses)
   *   radius: relative to the Sun's radius $R_{\odot}$ ($1 R_{\odot} \sim10$ Jupiter radii)

In [ ]:
starsed.results

In [ ]:
### example syntax to grab a specific parameter
starsed.radius #starsed.radius[0] to get the value only

#$\underline{\rm Resources\ \&\ Software}$:


### $\underline{\rm Databases\ }$:
$\bullet$ [Planet Hunters TESS](https://www.zooniverse.org/projects/nora-dot-eisner/planet-hunters-tess): Zooniverse project where you can help identify transiting exoplanet candidates discovered by the TESS mission -- no fancy tools required!

$\bullet$ [TESS Eclipsing Binary Catalog](https://tessebs.villanova.edu/): Catalog of EBs algorithmically identified from TESS light curves ([Prsa et al. 2022](https://ui.adsabs.harvard.edu/abs/2022yCat..22580016P/abstract)).


$\bullet$ [Kepler Eclipsing Binary Catalog](https://keplerebs.villanova.edu/): Catalog of EBs algorithmically identified from *Kepler* light curves ([Kirk et al. 2016](https://ui.adsabs.harvard.edu/abs/2016AJ....151...68K/abstract) \& previous papers).

$\bullet$ [Exoplanet Follow-up Observing Program (ExoFOP)](https://exofop.ipac.caltech.edu/tess/): "website designed to optimize resources and facilitate collaboration in follow-up studies of exoplanet candidates."

###  $\underline{\rm General\ Tools}$:
$\bullet$ [$\texttt{VARTOOLS}$](http://www.astro.princeton.edu/~jhartman/vartools.html), for creating synthetic light curves and analyzing light curves ([Hartman & Bakos 2016](https://ui.adsabs.harvard.edu/abs/2016A%26C....17....1H/abstract)).

### [$\underline{\rm Analytic\ Transit\ Model\ Fitting}$](https://ui.adsabs.harvard.edu/abs/2002ApJ...580L.171M/exportcitation):
#### $\textit{Python}$:
$\bullet$ [$\texttt{batman}$](https://lkreidberg.github.io/batman/docs/html/index.html): Python user-friendly transit model package ([Kreidberg 2015](https://ui.adsabs.harvard.edu/abs/2015PASP..127.1161K/abstract)).

$\bullet$ [$\texttt{exoplanet}$](https://github.com/exoplanet-dev/exoplanet): Python meta-module that bundles several other modules to fit transit/eclipse models, radial velocities (RVs), and astrometric orbits. Models correlated noise as a Gaussian Process with [$\texttt{celerite2}$](https://celerite2.readthedocs.io/en/latest/) \([Foreman-Mackey 2018](https://ui.adsabs.harvard.edu/abs/2018RNAAS...2...31F/abstract)) and calculates uncertainties using [$\texttt{PyMC}$](https://www.pymc.io/welcome.html)'s sophisticated Bayesian inference methods. [Decent documentation](https://docs.exoplanet.codes/en/latest/), but hard to install; feel free to contact Dan's research group for help if you try to use this.

#### $\textit{Non-Python}$:
$\bullet$ [$\texttt{EXOFASTv2}$](https://github.com/jdeast/EXOFASTv2): the workhorse for Dan's MESSy research group. IDL (not Python) software that can fit transit/eclipse (including transit timing variations), out-of-transit variability, radial velocities (of stars and planets!), SEDs, astrometry, Doppler Tomography/Rossiter-McLaughlin, etc. for many planets around many stars, correctly accounting for the light contributed by each star in each dataset  ([Eastman et al. 2019](https://ui.adsabs.harvard.edu/abs/2019arXiv190709480E/abstract)).

### $\underline{\rm Advanced\ Eclipse\ Model\ Fitting}$:
#### $\textit{Python}$:

$\bullet$ [$\texttt{PHysics Of Eclipsing BinariEs (PHOEBE)}$](https://phoebe-project.org): Models individual surface elements as triangles, then compute LCs, RVs, and SEDs. Commonly used by EB experts. Powerful, though computationally demanding; ([Prsa \& Zwitter (2005) \& subsequent papers](https://ui.adsabs.harvard.edu/search/q=docs(4cf2645878014c10dda7bcb48b493884)&sort=date%20desc%2C%20bibcode%20desc&p_=0).

$\bullet$ [$\texttt{Eclipsing binary Learning and Interactive System (ELISa)}$](https://github.com/mikecokina/elisa/): New module to model individual surface elements on each star, then compute LCs and RVs, including spots and pulsations. GitHub repository (previous link) contains a directory with tutorials. By [Cokina, Fedurco, \& Parimuch (2025)](https://ui.adsabs.harvard.edu/abs/2021A%26A...652A.156C/abstract).

$\bullet$ [$\texttt{allesfitter}$](https://www.allesfitter.com/): Frequently used meta-module using [$\texttt{ellc}$](https://github.com/pmaxted/ellc) \([Maxted 2016](https://ui.adsabs.harvard.edu/abs/2016A%26A...591A.111M/abstract)) for LCs plus modules for RVs, starspots, stellar flares, and Gaussian Processes ([Gunther \& Daylan 2021](https://ui.adsabs.harvard.edu/abs/2021ApJS..254...13G/abstract)).

#### $\textit{Non-Python}$:
$\bullet$ [$\texttt{JKTEBOP}$](https://www.astro.keele.ac.uk/jkt/codes/jktebop.html): FORTRAN77 code that is a modified version of the well-tested $\texttt{EBOP}$ ([Popper \& Etzel 1981](https://ui.adsabs.harvard.edu/abs/1981AJ.....86..102P/abstract)) code. Fits 1 LC and 1 RV dataset only. Updates described across several papers; see the [References](https://www.astro.keele.ac.uk/jkt/codes/jktebop.html) section of John Southworth's homepage.